In [1]:
from os import listdir
from os.path import join

DIR = './wiki-pages-text/'

data_files = listdir(DIR)
docs = {}  # {doc_title: [('sent_id': sent)]}

In [2]:
import pickle
from tqdm import tqdm

doc_id = 0

for file in tqdm(data_files):
    with open(join(DIR, file), 'r') as f:
        for line in f:
            title, sent_id, sent = line.strip('\n').split(' ', 2)
            docs[title] = docs.get(title, []) + [(sent_id, sent)]
print(str(len(docs)) + " files in corpus.")

100%|██████████| 109/109 [01:01<00:00,  1.77it/s]

5396106 files in corpus.


In [3]:
titles = {} # {doc_id, title}
contents = []

i = 0
for title, sents in tqdm(docs.items()):
    titles[i] = title
    i += 1
    content = ""
    for sent_id_, sent in sents:
        content += sent

    contents.append(content)

del docs # free memory used by docs

100%|██████████| 5396106/5396106 [00:19<00:00, 281099.11it/s]


In [4]:
with open('titles_dict.pkl', 'wb') as f_titles:
    pickle.dump(titles, f_titles)

# Find best matching documents

In [20]:
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from time import time

tf_idf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer())
])
tf_idf_matrix = tf_idf.fit_transform(contents)
print(tf_idf_matrix.shape)

(5396106, 4608465)


In [11]:
# with open('tf_idf_nltk.pkl', 'wb') as f_docs:
#     pickle.dump(tf_idf, f_docs)

OSError: [Errno 22] Invalid argument

In [24]:
claim1 = tf_idf.transform(["Nikolaj Coster-Waldau worked with the Fox Broadcasting Company."])

In [39]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = [ similarity[0] for similarity in cosine_similarity(tf_idf_matrix, claim1)]
doc_ids = np.argsort(cosine_similarities)[-50:][::-1]

In [41]:
for doc_id in doc_ids:
    print("title: ", titles[doc_id])

title:  Coster
title:  Ved_verdens_ende
title:  Waldau_-LRB-surname-RRB-
title:  New_Amsterdam_-LRB-TV_series-RRB-
title:  A_Second_Chance_-LRB-2014_film-RRB-
title:  Coster_-LRB-disambiguation-RRB-
title:  Nukaaka_Coster-Waldau
title:  Sven_Coster
title:  Kalle_Coster
title:  A_Thousand_Times_Good_Night
title:  Jean-Baptiste_De_Coster_-LRB-Jesuit-RRB-
title:  Nikolaj_Coster-Waldau
title:  De_Coster
title:  The_Baker_-LRB-film-RRB-
title:  Tracy_Coster
title:  Saskia_De_Coster
title:  The_Other_Woman_-LRB-2014_film-RRB-
title:  Project_Laurens_Janszoon_Coster
title:  Dick_Coster
title:  Salomon_Coster
title:  Fox_-LRB-Turkey-RRB-
title:  Shot_Caller_-LRB-film-RRB-
title:  Harry_Waldau
title:  Arnold_Coster
title:  Small_Crimes
title:  Kunsthallen_Nikolaj
title:  Nikolaj_Plads
title:  Fox_Lab
title:  Gustav_Waldau
title:  Lionel_Lockridge
title:  45_-LRB-film-RRB-
title:  Samuel_Coster
title:  Stan_Coster
title:  Fox_House
title:  Willem_Jacobszoon_Coster
title:  Tommy_Coster
title:  Co